<img src="Avenida máxima probable. P.H. Norte II.jpg">

[Fit surface to polynomials in Python](https://gist.github.com/amroamroamro/1db8d69b4b65e8bc66a6)

In [1]:
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
a = 3.5465
b = 0.208
c = 0.7727
d = -0.0315

In [ ]:
def Q_from_TS(T, S, a=3.5465, b=0.208, c=0.7727, d=-0.0315):
    """"""
    
    Q = a * T**b * S**(c + d * np.log10(T))
    
    return Q

In [ ]:
T = 10 # años
S = 270 # km²

In [ ]:
Q = Q_from_TS(T, S)
print('Q = {0:.1f} m³/s'.format(Q))

In [4]:
Q=363
S=270

In [5]:
def f(T, S=S, Q=Q, a=3.5465, b=0.208, c=0.7727, d=-0.0315):
    return round(a * T**b * S**(c + d * np.log10(T)) - Q, 1)

In [6]:
def fprime(T, S=S, Q=Q, a=3.5465, b=0.208, c=0.7727, d=-0.0315):
    return a * b * T**(b-1) * S**(c - d * np.log10(T)) + a * T**b * (d / (np.log(10) * T)) * S**(c +d * np.log10(T) - 1)

In [7]:
# calcular retorno por el método de Brent
Tbrent = optimize.root_scalar(f, bracket=[0.001, 1000], method='brentq').root

In [8]:
# calcular retorno por el método de Newton
Tnewton = optimize.root_scalar(f, x0=2, fprime=fprime, method='newton').root

In [9]:
print('Brent:\tT = {0:.1f} años'.format(Tbrent))
print('Newton:\tT = {0:.1f} años'.format(Tnewton))

Brent:	T = 10.0 años
Newton:	T = 10.0 años


In [ ]:
T = 10

In [ ]:
Ss = np.logspace(0, 4, num=30)
Ts = [5, 10, 25, 50, 100]

Qs = pd.DataFrame(index=Ss, columns=Ts)
for T in Qs.columns:
    Qs[T] = np.array([Q_from_TS(T, S) / S for S in Ss])

In [ ]:
plt.figure(figsize=(16, 4))

for T in Qs.columns:
    plt.plot(Ss, Qs[T], label=T)

yticks = np.linspace(0, 10, 11)
plt.yticks(ticks=yticks, labels=yticks)
plt.xlabel('S [km²]')
plt.ylabel('Qesp [m³/(s·km²)]')
plt.yscale('log')
plt.xscale('log')
plt.legend();

In [ ]:
abaco = pd.read_excel('Periodos_retorno.xlsx', sheet_name='AMP', index_col='S')

In [ ]:
# ajustar polinomio
pars = np.polyfit(np.log10(abaco.index), np.log10(abaco.Q5), deg=3)
print('log Qs = {0:.3f} (log S)³ + {1:.3f} (log S)² + {2:.3f} log S + {3:.3f}'.format(*pars))

In [ ]:
# calcular puntos de la curva ajustada
S_ = np.logspace(0, 5, 100)
Q_ = 10**np.polyval(pars, np.log10(S_))

In [ ]:
# gráfico
plt.scatter(abaco.index, abaco.Q5, s=10, c='k', label='abaco')
plt.plot(S_, Q_)

plt.xscale('log')
plt.yscale('log')

***

In [ ]:
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [ ]:
# some 3-dim points
mean = np.array([0.0,0.0,0.0])
cov = np.array([[1.0,-0.5,0.8], [-0.5,1.1,0.0], [0.8,0.0,1.0]])
data = np.random.multivariate_normal(mean, cov, 50)

In [ ]:
data

In [ ]:
# regular grid covering the domain of the data
XX,YY = np.meshgrid(np.arange(-3.0, 3.0, 0.5), np.arange(-3.0, 3.0, 0.5))
X = XX.flatten()
Y = YY.flatten()

In [ ]:
order = 2    # 1: linear, 2: quadratic
if order == 1:
    # best-fit linear plane: Z = a·X + b·Y + c
    A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
    pars, residues, rank, s = scipy.linalg.lstsq(A, data[:,2])    # coefficients
    
    # evaluate it on grid
    a, b, c = pars
    ZZ = a * XX + b * YY + c
    
    # or expressed using matrix/vector product
    #Z = np.dot(np.c_[XX, YY, np.ones(XX.shape)], C).reshape(X.shape)

elif order == 2:
    # best-fit quadratic curve: Z = a + b·X + c·Y + d·X·Y + e·X² + f·Y²
    A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2]
    pars, residues, rank, s = scipy.linalg.lstsq(A, data[:,2])
    
    # evaluate it on a grid
    a, b, c, d, e, f = pars
    ZZ = np.dot(np.c_[np.ones(X.shape), X, Y, X*Y, X**2, Y**2], pars).reshape(XX.shape)

In [ ]:
plt.figure(figsize=(5, 5))
plt.scatter(XX, YY, s=10, c=ZZ, cmap='viridis')
plt.scatter(data[:,0], data[:,1], marker='x', c=data[:,2], cmap='viridis')
plt.xlabel('X')
plt.ylabel('Y')
plt.axis('equal');

In [ ]:
# Z_ = np.polynomial.polynomial.polyval2d(X, Y, [pars[1], pars[2], pars[0]])

In [ ]:
# plot points and fitted surface
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_surface(XX, YY, ZZ, rstride=1, cstride=1, alpha=0.2)
ax.scatter(data[:,0], data[:,1], data[:,2], c='r', s=50)
plt.xlabel('X')
plt.ylabel('Y')
ax.set_zlabel('Z')
# ax.axis('equal')
ax.axis('tight')
plt.show()

In [ ]:
pars